# Transformers 를 이용해 Backbone 사용

## Transformers 설치
- `pip install transformers`

### Tokenizer, Model Loading
- Huggingface 모델 허브에서 제공하는 처리 모델을 다운받아 로딩한다.
- 다운로드된 모델은 `사용자 home 디렉토리\.cache\huggingface` 에 저장된다.
- 미리 학습된 언어 모델을 다운받아 사용할 때는 그 언어모델이 사용한 tokenizer를 같이 받아서 사용한다.

### [Auto Classes](https://huggingface.co/docs/transformers/model_doc/auto)
- Huggingface 에서 제공하는 다양한 모델들은 손쉽게 불러오고 사용할 수 있도록 설계된 유틸리티 클래스들을 말한다.
- 미리 학습된 특정 모델의 이름(모델 허브상에서 제공되는 이름)이나 저장된 local 경로를 제공하면 해당 모델에 맞는 적절한 클래스와 구성 요소를 자동으로 로드한다.
- 사용자는 모델을 사용하기 위한 정확한 클래스를 몰라도 쉽게 다양한 종류의 모델을 사용할 수있다.

#### 주요 Auto Class
- 기본 모델 Loading
    1. **AutoModel**
       - 주어진 모델 이름에 맞는 사전 학습된 모델을 자동으로 로드한다.
       - 예: `AutoModel.from_pretrained("bert-base-uncased")`: BERT 모델을 로드한다.
    2. **AutoTokenizer**
       - 해당 모델에 적합한 토크나이저를 자동으로 로드한다.
       - 예: `AutoTokenizer.from_pretrained("bert-base-uncased")`: BERT 모델에 맞는 토크나이저를 로드한다.
    3. **AutoConfig**
       - 모델의 설정(config)을 자동으로 로드한다. 모델 설정에는 모델의 하이퍼파라미터와 모델 구조 정보가 포함된다. 이 설정을 이용해 모델을 생성할 수있다.
       - 예: `AutoConfig.from_pretrained("bert-base-uncased")`
- Task 처리 모델 Loading
    - Pretrained backbone 모델에 각 task 에 맞는 estimator layer를 추가한 모델을 생성해 제공한다.
    - 주요 모델들
        1. **AutoModelForSequenceClassification**
           - 시퀀스(Text) 분류 작업을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")`
        2. **AutoModelForQuestionAnswering**
           - 질문-응답 작업을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")`
        3. **AutoModelForTokenClassification**
           - 토큰 분류 작업(예: 개체명 인식)을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForTokenClassification.from_pretrained("bert-base-uncased")`

In [1]:
from transformers import AutoModel, AutoTokenizer, AutoConfig

In [9]:
# model_id = "bert-base-uncased"
model_id = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
print(type(tokenizer))

model = AutoModel.from_pretrained(model_id)
print(type(model))

config = AutoConfig.from_pretrained(model_id)
print(type(config))

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\Playdata\anaconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

<class 'transformers.models.gpt2.tokenization_gpt2_fast.GPT2TokenizerFast'>


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

<class 'transformers.models.gpt2.modeling_gpt2.GPT2Model'>
<class 'transformers.models.gpt2.configuration_gpt2.GPT2Config'>


In [10]:
raw_txt = "I am a boy."
token = tokenizer(
    raw_txt,
    return_tensors="pt" # 토큰화 결과를 torch.Tensor으로 반환 (default는 list임)
)
token
# input_ids: 토큰 id
# attention_mask: input_ids의 각 토큰들이 실제 토큰인지 [PAD] 토큰인지 여부

{'input_ids': tensor([[  40,  716,  257, 2933,   13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [11]:
type(token['input_ids'])

torch.Tensor

In [12]:
model

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [ ]:
context_vector = model(**token)
# context_vector = model(input_ids=token['input_ids'], attention_mask=token['attention_mask'])

In [ ]:
context_vector.last_hidden_state.shape
# torch.Size([1, 5, 768])
# 1: 문장(문서) 개수, 5: 토큰 수, 768: embedding vector 차원

torch.Size([1, 5, 768])

In [ ]:
# tokenizer 확인
t = tokenizer.encode("I am a boy.") # token id만 리턴

In [18]:
tokenizer.decode(t)

'I am a boy.'

In [19]:
# 개별 토큰 확인
tokenizer.convert_tokens_to_ids("am")

321

In [20]:
tokenizer.convert_ids_to_tokens(321)

'am'

## kcbert
- BERT 모델을 한글 텍스트로 학습 시킨 Pretrained model.
    - BERT는 Transformer의 Encoder 부분을 이용해 구현된 언어모델
    - https://arxiv.org/abs/1810.04805 
- https://huggingface.co/beomi/kcbert-base

### 토크나이저 모델 load

In [21]:
from transformers import AutoModel, AutoTokenizer

model_id = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

In [22]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30000, 768, padding_idx=0)
    (position_embeddings): Embedding(300, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

### 입력값 토큰화

In [39]:
sentences = [
    "안녕",
    "Hugging Face는 인공지능(AI)과 자연어 처리(NLP) 분야에서 혁신적인 도구와 모델을 제공하는 AI 스타트업이다.",
    "2016년에 설립된 이 회사는 주로 오픈소스 라이브러리와 사전 학습된 NLP 모델을 제공을 제공한다."
]

In [ ]:
token = tokenizer(sentences[0])
token
# input_ids: 토큰 id들
# token_type_ids: 입력이 두 종류의 문장(문서)으로 구성된 경우, 각 토큰이 어떤 종류인지 구분
# attention_mask: input_ids의 개별 토큰들이 실제 값인지 아니면 [PAD]인지 구분

{'input_ids': [2, 19017, 3], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [41]:
token_list = tokenizer(
    sentences,
    max_length=10, # padding(추가하기)과 truncation(자르기)의 기준 길이 지정
    padding=True,
    truncation=True,
    return_tensors="pt"
)

In [43]:
token_list['input_ids']

tensor([[    2, 19017,     3,     0,     0,     0,     0,     0,     0,     0],
        [    2,    41,  4223,  4403,  4403, 18940,    39,  4243,  4773,     3],
        [    2, 26182,  4113, 20684,  4130,  2451, 22088, 20002, 15999,     3]])

In [44]:
token_list['attention_mask']

tensor([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [45]:
token_list['token_type_ids']

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

### BERT 모델을 이용해 context vector 추출
#### Model 추론결과
- **last_hidden_state**
    - 모든 token들에 대한 feature
    - 출력이 **many**인 작업에 사용한다.
- **pooler_output**
    - 입력 문장, 텍스트에 대한 context vector 이다.
    - 이 값은 **문장을 입력받아 처리하는 task**(ex: 문서분류-감정분석,문장카테고리분류, 문장유사도 분석)의 입력으로 사용한다.

In [46]:
output = model(**token_list)

In [47]:
output.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [48]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-4.3785e-01, -1.1922e+00,  1.9459e+00,  ...,  4.2119e-01,
           9.8295e-01,  1.0119e+00],
         [-3.0053e-01, -1.1015e+00,  9.0584e-01,  ...,  6.9805e-01,
           6.8852e-01,  9.2259e-01],
         [-8.9004e-01, -6.8068e-01,  8.9948e-01,  ...,  6.0443e-01,
           7.3096e-01, -3.4560e-01],
         ...,
         [-1.0742e+00, -1.8177e-01,  2.3927e+00,  ...,  1.9575e+00,
           3.9581e-01,  9.9546e-01],
         [-9.4360e-01, -2.6620e-01,  2.2696e+00,  ...,  1.5724e+00,
           3.0022e-01,  3.3455e-01],
         [-8.0589e-01, -5.7948e-01,  2.3011e+00,  ...,  1.8778e+00,
          -2.1813e-01, -4.0344e-01]],

        [[ 2.0945e-01, -7.9768e-01,  9.5041e-01,  ..., -3.0953e-01,
           1.0519e+00,  2.2225e-01],
         [-1.0890e+00,  3.9076e-02,  1.5402e+00,  ..., -1.2150e+00,
          -4.4001e-01, -9.4009e-02],
         [-2.3940e-01,  9.3101e-01,  8.0560e-01,  ..., -3.4402e-01,
           1.

In [49]:
output['pooler_output'].shape

torch.Size([3, 768])